<a href="https://colab.research.google.com/github/MicaelaRomeroC3/COPs/blob/main/03_Tablafechas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
ruta_historical = '/content/drive/MyDrive/COPS/DATA/3.HISTORICAL/llamadas.parquet'
if os.path.exists(ruta_historical):
    df = pd.read_parquet(ruta_historical)
    print(f"📊 Histórico cargado con {len(df)} registros.")
else:
    print("⚠️ No se encontró el archivo histórico. Asegúrate de ejecutarlo antes.")
    exit()

df_grouped = df.groupby('FECHA_EVALUACIÓN').agg(
    P1_percent=('P1', lambda x: round((x.mean()) * 100, 2)),
    P2_percent=('P2', lambda x: round((x.mean()) * 100, 2)),
    P3_percent=('P3', lambda x: round((x.mean()) * 100, 2)),
    P4_percent=('P4', lambda x: round((x.mean()) * 100, 2)),
    P5_percent=('P5', lambda x: round((x.mean()) * 100, 2)),
    P6_percent=('P6', lambda x: round((x.mean()) * 100, 2)),
    S1_percent=('S1', lambda x: round((x.mean()) * 100, 2)),
    S2_percent=('S2', lambda x: round((x.mean()) * 100, 2)),
    S3_percent=('S3', lambda x: round((x.mean()) * 100, 2)),
    S4_percent=('S4', lambda x: round((x.mean()) * 100, 2)),
    S5_percent=('S5M', lambda x: round((x.mean()) * 100, 2)),
    C1_percent=('C1M', lambda x: round((x.mean()) * 100, 2)),
    C2_percent=('C2M', lambda x: round((x.mean()) * 100, 2)),
    T1_percent=('T1M', lambda x: round((x.mean()) * 100, 2)),
    T2_percent=('T2M', lambda x: round((x.mean()) * 100, 2)),
    T3_percent=('T3', lambda x: round((x.mean()) * 100, 2))
).reset_index()
df['total_registros'] = df.groupby('FECHA_EVALUACIÓN')['FECHA_EVALUACIÓN'].transform('count')
total_registros = df.groupby('FECHA_EVALUACIÓN')['total_registros'].max()
df_grouped['total_registros'] = total_registros.values
df_grouped['total_registros'] = df_grouped['total_registros'].astype(int)
df_transposed = df_grouped.set_index('FECHA_EVALUACIÓN').transpose()

detalle_dict = {
    'P1_percent': 'PRESENTACIÓN, SALUDO CORPORATIVO Y GRABACIÓN DE LA LLAMADA',
    'P2_percent': 'IDENTIFICACIÓN DEL INTERLOCUTOR - LOPD',
    'P3_percent': 'ESCUCHA COMPRENSIVA Y EMPÁTICA',
    'P4_percent': 'TRATO CORDIAL AL CLIENTE',
    'P5_percent': 'RESPONSABILIDAD CORPORATIVA',
    'P6_percent': 'DESPEDIDA CORPORATIVA',
    'S1_percent': 'INFORMACIÓN MOTIVO DE LA LLAMADA',
    'S2_percent': 'SITUACIÓN ACTUAL DEL DEUDOR',
    'S3_percent': 'SONDEO: IDENTIFICAR CAPACIDAD DE PAGO Y DETECCIÓN DE NECESIDADES',
    'S4_percent': 'DIRECCIÓN',
    'S5_percent': 'PREPARACIÓN CIERRE : RESUMEN DE ACUERDOS',
    'C1_percent': 'OFRECER TPV COMO ALTERNATIVA PRIORITARIA DE PAGO',
    'C2_percent': 'PRIORIZA LOS CANALES DIGITALES U OTROS MEDIOS PARA REALIZAR LOS INGRESOS LO ANTES POSIBLE',
    'T1_percent': 'USO EFECTIVO DEL SCRIPT',
    'T2_percent': 'CAPACIDAD DE EXPRESIÓN',
    'T3_percent': 'BUSCAR EL COMPROMISO DE PAGO ENUNCIANDO LOS BENEFICIOS Y LAS CONSECUENCIAS DE LA REGULARIZACIÓN'
}

# Renombrar las filas (índice) del DataFrame transpuesto
df_transposed.insert(0, 'DETALLE', df_transposed.index.map(detalle_dict))

# Diccionario AGRUPADOR
agrupador_dict = {
    'P1_percent': 'PRESENTACIÓN',
    'P2_percent': 'PRESENTACIÓN',
    'P3_percent': 'PRESENTACIÓN',
    'P4_percent': 'PRESENTACIÓN',
    'P5_percent': 'PRESENTACIÓN',
    'P6_percent': 'PRESENTACIÓN',
    'S1_percent': 'SONDEO',
    'S2_percent': 'SONDEO',
    'S3_percent': 'SONDEO',
    'S4_percent': 'SONDEO',
    'S5_percent': 'SONDEO',
    'C1_percent': 'CANALIZACION',
    'C2_percent': 'CANALIZACION',
    'T1_percent': 'SCRIPT',
    'T2_percent': 'SCRIPT',
    'T3_percent': 'SCRIPT',
    'total_registros': 'TOTAL'
}

# Insertar columna AGRUPADOR
df_transposed.insert(0, 'AGRUPADOR', df_transposed.index.map(agrupador_dict))

# Ponderaciones
ponderacion_dict = {
    'P1_percent': 0.02,
    'P2_percent': 0.10,
    'P3_percent': 0.03,
    'P4_percent': 0.03,
    'P5_percent': 0.05,
    'P6_percent': 0.02,
    'S1_percent': 0.03,
    'S2_percent': 0.02,
    'S3_percent': 0.10,
    'S4_percent': 0.10,
    'S5_percent': 0.05,
    'C1_percent': 0.10,
    'C2_percent': 0.10,
    'T1_percent': 0.05,
    'T2_percent': 0.10,
    'T3_percent': 0.10
}

df_transposed.insert(0, 'PONDERACION', df_transposed.index.map(ponderacion_dict))

df_transposed.loc[df_transposed.index == 'GLOBAL', 'PONDERACION'] = ''


📊 Histórico cargado con 10000 registros.


/tmp/ipython-input-2972139022.py:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_transposed.loc[df_transposed.index == 'GLOBAL', 'PONDERACION'] = ''


In [ ]:
## Para calcular los % de las categorias agrupadas

df_presentacion = df_transposed[df_transposed['AGRUPADOR'].str.upper().str.strip() == 'PRESENTACIÓN'].copy()

# Paso 2: Asegúrate de que la columna PONDERACION es float (en formato 0.02, 0.10, etc)
df_presentacion['PONDERACION'] = (
    df_presentacion['PONDERACION']
    .astype(str)
    .str.replace('%', '', regex=False)
    .str.replace(',', '.', regex=False)
    .astype(float) / 100
)

# Paso 3: Seleccionar solo las columnas de fechas
columnas_fechas = [col for col in df_transposed.columns if col not in ['INDICADOR', 'AGRUPADOR', 'PONDERACION', 'DETALLE']]


# Paso 4: Calcular la ponderación * valor para cada fila
ponderados = df_presentacion[columnas_fechas].multiply(df_presentacion['PONDERACION'].values, axis=0)

# Paso 5: Sumar los valores ponderados por columna
suma_ponderada = ponderados.sum(axis=0)

# Paso 6: Dividir entre 0.25 (peso total del grupo “Presentación”)
resultado_presentacion = (suma_ponderada / 0.25)*100

# Paso 7: Crear nueva fila con resultado
fila_presentacion = pd.DataFrame([['PRESENTACIÓN', 'PRESENTACIÓN', '', *resultado_presentacion]],
                                 columns=['INDICADOR', 'AGRUPADOR', 'PONDERACION'] + columnas_fechas)

# Paso 8: Insertar la nueva fila al inicio del dataframe
df_transposed = pd.concat([fila_presentacion, df_transposed], ignore_index=True)

def agregar_indicador_grupo(df_transposed, nombre_grupo, peso_grupo):
    # Paso 1: Filtrar el grupo (en mayúsculas y sin espacios)
    df_grupo = df_transposed[df_transposed['AGRUPADOR'].str.upper().str.strip() == nombre_grupo.upper()].copy()

    # Paso 2: Asegurar que PONDERACION está en float
    df_grupo['PONDERACION'] = (
        df_grupo['PONDERACION']
        .astype(str)
        .str.replace('%', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float) / 100
    )

    # Paso 3: Seleccionar solo columnas de fechas
    columnas_fechas = [col for col in df_transposed.columns if col not in ['INDICADOR', 'AGRUPADOR', 'PONDERACION', 'DETALLE']]

    # Paso 4: Asegurar que los valores de fechas están en float
    df_grupo[columnas_fechas] = df_grupo[columnas_fechas].apply(
        lambda col: col.astype(str)
                     .str.replace('%', '', regex=False)
                     .str.replace(',', '.', regex=False)
                     .astype(float) / 100
    )

    # Paso 5: Multiplicar cada fila por su ponderación
    ponderados = df_grupo[columnas_fechas].multiply(df_grupo['PONDERACION'].values, axis=0)

    # Paso 6: Sumar por columna
    suma_ponderada = ponderados.sum(axis=0)

    # Paso 7: Dividir entre el peso total del grupo
    resultado = ((suma_ponderada / peso_grupo) )* 100

    # Paso 8: Crear fila de resultado
    fila = pd.DataFrame([[nombre_grupo.upper(), nombre_grupo.upper(), '', *resultado]],
                        columns=['INDICADOR', 'AGRUPADOR', 'PONDERACION'] + columnas_fechas)

    # Paso 9: Agregar fila al inicio
    return pd.concat([fila, df_transposed], ignore_index=True)

df_transposed = agregar_indicador_grupo(df_transposed, 'SONDEO', 0.003)
df_transposed = agregar_indicador_grupo(df_transposed, 'SCRIPT', 0.0025)
df_transposed = agregar_indicador_grupo(df_transposed, 'CANALIZACION', 0.0020)

# 1️⃣ Ordenar filas según el orden deseado en 'INDICADOR'
orden_indicadores = ['PRESENTACIÓN', 'SONDEO', 'CANALIZACION', 'SCRIPT']
df_transposed['INDICADOR'] = df_transposed['INDICADOR'].astype(str)
df_transposed['orden_temp'] = df_transposed['INDICADOR'].apply(
    lambda x: orden_indicadores.index(x.upper()) if x.upper() in orden_indicadores else len(orden_indicadores)
)
df_transposed = df_transposed.sort_values(by='orden_temp').drop(columns='orden_temp').reset_index(drop=True)

# 2️⃣ Reordenar las columnas para que DETALLE esté justo después de AGRUPADOR
columnas_ordenadas = ['INDICADOR', 'AGRUPADOR', 'DETALLE', 'PONDERACION'] + [
    col for col in df_transposed.columns if col not in ['INDICADOR', 'AGRUPADOR', 'DETALLE', 'PONDERACION']
]
df_transposed = df_transposed[columnas_ordenadas]

# Crear la ruta completa
ruta_salida = '/content/drive/MyDrive/COPS/OUTPUT/TABLA_FECHAS.xlsx'

# Crear la carpeta si no existe
os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)

# Suponemos que las fechas empiezan desde la columna índice 4 (columna 5 visualmente)
columnas_fechas = df_transposed.columns[4:]

# Copiamos y multiplicamos por la ponderación
df_transposed_copy = df_transposed.copy()

for col in columnas_fechas:
    df_transposed_copy[col] = df_transposed_copy[col].astype(float) * pd.to_numeric(df_transposed_copy['PONDERACION'], errors='coerce').fillna(0)

# Sumamos para obtener la fila GLOBAL
fila_global = df_transposed_copy[columnas_fechas].sum()

# Creamos la nueva fila
nueva_fila = pd.Series(index=df_transposed.columns, dtype='object')
nueva_fila['INDICADOR'] = 'GLOBAL'
nueva_fila['AGRUPADOR'] = 'GLOBAL'
nueva_fila['DETALLE'] = ''
nueva_fila['PONDERACION'] = ''
for col in columnas_fechas:
    nueva_fila[col] = fila_global[col]

# Insertamos la fila GLOBAL antes de la fila PRESENTACIÓN
idx_presentacion = df_transposed[df_transposed['INDICADOR'] == 'PRESENTACIÓN'].index[0]
df_transposed = pd.concat([
    df_transposed.iloc[:idx_presentacion],
    pd.DataFrame([nueva_fila]),
    df_transposed.iloc[idx_presentacion:]
]).reset_index(drop=True)

In [ ]:
# Guardar el DataFrame transpuesto en el archivo Excel
with pd.ExcelWriter(ruta_salida, engine='openpyxl') as writer:
    df_transposed.to_excel(writer, sheet_name='TABLA_FECHAS', index=False)

print(f"✅ El archivo ha sido guardado correctamente en: {ruta_salida}")

✅ El archivo ha sido guardado correctamente en: /content/drive/MyDrive/COPS/OUTPUT/TABLA_FECHAS.xlsx
